In [1]:
import os
import numpy as np
import cv2
import math
import seaborn as sns
import matplotlib.pyplot as plt

def ci95(inp):
    max95 = np.mean(inp) + (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    min95 = np.mean(inp) - (1.96 * (np.std(inp) / math.sqrt(len(inp))))
    return min95, max95

def dice_coef_score(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
#     print(intersection)
    return (2. * intersection) / (np.sum(y_true_f) + np.sum(y_pred_f))

def IoU(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
#     print(intersection)

    iou = intersection/(np.sum(y_true_f) + np.sum(y_pred_f)-intersection)
    return round(np.mean(iou), 2)

In [ ]:
name_li = ['brmh','kyuh','gugh']
# cutoff_values=[0.1,0.2,0.3]
cutoff_values=[0.05]

for c, cutoff_value in enumerate(cutoff_values):
    print('-'*20)
    print('cutoff : {}'.format(cutoff_value))
    print('-'*20)
    for name in name_li:
        print(name+'_respiratory')
        gt_base = 'D:/lunit/01_data_analysis/data/img/result/{}_respiratory/gt_mask'.format(name)
        base = 'D:/lunit/01_data_analysis/data/img/ai_json(all_files)/{}_respiratory/ai_mask'.format(name)
        ai_mask_list = os.listdir(base)
        sum_image = len(ai_mask_list)

        fppi_cnt = 0
        fn_cnt = 0
        tp_cnt = 0

        total_cnt=0
        gt_ims= []
        ai_ims = []
        true_detection_rate_li = []
        dscs=[]
        for index, ai_mask in enumerate(sorted(ai_mask_list)):
    #         if index==10:
    #             break
            tp = 0
            fn = 0
            gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
            gt_im = cv2.imread(os.path.join(gt_base, gt_file))
            w,h,_=gt_im.shape
            im = cv2.imread(os.path.join(base, ai_mask))
            im = cv2.resize(im, (h,w),interpolation=cv2.INTER_LINEAR)
    #         print(gt_file, ai_mask)
    #         print(gt_im.shape, im.shape)

            imgray = cv2.cvtColor(gt_im,cv2.COLOR_BGR2GRAY)
            ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
            gt_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
            ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
            ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #         print(len(gt_contours), len(ai_contours))
            total_cnt+=len(ai_contours)
        #     print(gt_im.shape, im.shape)
            if gt_im.sum() == 0: # normal
                fppi_cnt += len(ai_contours)

            elif im.sum()==0:
                fn_cnt += len(gt_contours)
                fn+=1

            else:
                for ai_contour in ai_contours:
                    _image = np.zeros_like(gt_im)
                    _image_ai = cv2.drawContours(_image, [ai_contour], 0, color=1, thickness=-1)

                    dsc_li=[]
                    for gt_contour in gt_contours:
                        _image = np.zeros_like(gt_im)
                        _image_gt = cv2.drawContours(_image, [gt_contour], 0, color=1, thickness=-1)
                        dsc_li.append(dice_coef_score(_image_gt, _image_ai))
                    
                    if max(dsc_li) < cutoff_value:
                        fppi_cnt += 1
                    else:
#                         if max(dsc_li)>0:
                        tp_cnt+=1
    
                if (c==2 and max(dsc_li)>0):
                    dscs.append(max(dsc_li))
                
                for gt_contour in gt_contours:
                    _image = np.zeros_like(gt_im)
                    _image_gt = cv2.drawContours(_image, [gt_contour], 0, color=1, thickness=-1)

                    dsc_li=[]
                    for ai_contour in ai_contours:
                        _image = np.zeros_like(gt_im)
                        _image_ai = cv2.drawContours(_image, [ai_contour], 0, color=1, thickness=-1)
                        dsc_li.append(dice_coef_score(_image_ai, _image_gt))

                    if max(dsc_li) < cutoff_value:
                        fn_cnt += 1

        fppi = fppi_cnt/sum_image
        print('='*20)
        print('Calculated FPPI: {:0.2f}'.format(fppi))
        print('Total count: {}'.format(total_cnt))
        print('TP count: {}'.format(tp_cnt))
        print('FP count: {}'.format(fppi_cnt))
        print('FN count: {}'.format(fn_cnt))
        print('Sensitivity: {:0.2f}'.format(tp_cnt/(tp_cnt+fn_cnt)))
        print('='*20)

--------------------
cutoff : 0.05
--------------------
brmh_respiratory


In [ ]:
name_li = ['brmh','kyuh','gugh']

for name in name_li:
    print(name+'_respiratory')
    gt_base = 'D:/lunit/01_data_analysis/data/img/result/{}_respiratory/gt_mask'.format(name)
    base = 'D:/lunit/01_data_analysis/data/img/ai_json(all_files)/{}_respiratory/ai_mask'.format(name)
    ai_mask_list = os.listdir(base)
    sum_image = len(ai_mask_list)

    fppi_cnt = 0
    fn_cnt = 0
    tp_cnt = 0

    total_cnt=0
    gt_ims= []
    ai_ims = []
    true_detection_rate_li = []
    dscs=[]
    for index, ai_mask in enumerate(sorted(ai_mask_list)):
#         if index==10:
#             break
        tp = 0
        fn = 0
        gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
        gt_im = cv2.imread(os.path.join(gt_base, gt_file))
        w,h,_=gt_im.shape
        im = cv2.imread(os.path.join(base, ai_mask))
        im = cv2.resize(im, (h,w),interpolation=cv2.INTER_LINEAR)
#         print(gt_file, ai_mask)
#         print(gt_im.shape, im.shape)

        imgray = cv2.cvtColor(gt_im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        gt_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         print(len(gt_contours), len(ai_contours))
        total_cnt+=len(ai_contours)
    #     print(gt_im.shape, im.shape)
        if gt_im.sum() == 0: # normal
            fppi_cnt += len(ai_contours)

        elif im.sum()==0:
            fn_cnt += len(gt_contours)
            fn+=1

        else:
            for ai_contour in ai_contours:
                _image = np.zeros_like(gt_im)
                _image_ai = cv2.drawContours(_image, [ai_contour], 0, color=1, thickness=-1)

                dsc_li=[]
                for gt_contour in gt_contours:
                    _image = np.zeros_like(gt_im)
                    _image_gt = cv2.drawContours(_image, [gt_contour], 0, color=1, thickness=-1)
                    dsc_li.append(dice_coef_score(_image_gt, _image_ai))

            if max(dsc_li)>0:
                dscs.append(max(dsc_li))

    draw_plot(dscs, name)
    
    
    
    
#     # Draw the density plot
#     sns_plt = sns.distplot(dscs, hist = False, kde = True,
#          kde_kws = {'linewidth': 3},color = 'black')

#         # Plot formatting
#         # plt.axis('off')

#     plt.xlabel('Dice Similarity Coefficient')
#     plt.ylabel('TP count')
#     sns_plt.figure.savefig("{}.tif".format(name), dpi=300)


In [ ]:
from scipy.interpolate import BSpline, make_interp_spline

def draw_plot(dscs, name):
    n, bins, patches = plt.hist(dscs, bins=5)
    plt.clf()

    x=[0]
    for i in range(len(bins)-1):
        a=bins[i]
        b=bins[i+1]
        x.append((a+b)/2)
    x.append(1) 

    y=np.append([0], n)
    y=np.append(y, [0])


    x=np.array(x)
#     print(x, y)
    xnew = np.linspace(x.min(), x.max(), 200) 

    #define spline
    spl = make_interp_spline(x, y, k=2)
    y_smooth = spl(xnew)
    
#     sns.distplot(dscs, bins=5)
#     plt.tight_layout()
#     plt.show()
    
#     plt.plot(x, y)
#     plt.xlim([0,1])
#     plt.ylim([0,max(n)+20])
#     plt.show()

#     #create smooth line chart 
    plt.plot(xnew, y_smooth, 'k',linewidth=2)
    plt.xlim([0,1])
    plt.ylim([0,350])
    plt.xlabel('Dice Similarity Coefficient')
    plt.ylabel('TP count')
    plt.tight_layout()
    plt.savefig('{}_fig.tif'.format(name), dpi=300)
    plt.show()

In [106]:
name_li = ['brmh','kyuh','gugh']

dscs1=[]
dscs2=[]
dscs3=[]

for name in name_li:
    print(name+'_respiratory')
    gt_base = 'D:/lunit/01_data_analysis/data/img/result/{}_respiratory/gt_mask'.format(name)
    base = 'D:/lunit/01_data_analysis/data/img/ai_json(all_files)/{}_respiratory/ai_mask'.format(name)
    ai_mask_list = os.listdir(base)
    sum_image = len(ai_mask_list)

    fppi_cnt = 0
    fn_cnt = 0
    tp_cnt = 0

    total_cnt=0
    gt_ims= []
    ai_ims = []
    true_detection_rate_li = []
    
    for index, ai_mask in enumerate(sorted(ai_mask_list)):
#         if index==10:
#             break
        tp = 0
        fn = 0
        gt_file = '{}_mask.png'.format(ai_mask.split('_cutoff')[0])
        gt_im = cv2.imread(os.path.join(gt_base, gt_file))
        w,h,_=gt_im.shape
        im = cv2.imread(os.path.join(base, ai_mask))
        im = cv2.resize(im, (h,w),interpolation=cv2.INTER_LINEAR)
#         print(gt_file, ai_mask)
#         print(gt_im.shape, im.shape)

        imgray = cv2.cvtColor(gt_im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        gt_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
        ai_contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         print(len(gt_contours), len(ai_contours))
        total_cnt+=len(ai_contours)
    #     print(gt_im.shape, im.shape)
        if gt_im.sum() == 0: # normal
            fppi_cnt += len(ai_contours)

        elif im.sum()==0:
            fn_cnt += len(gt_contours)
            fn+=1

        else:
            for ai_contour in ai_contours:
                _image = np.zeros_like(gt_im)
                _image_ai = cv2.drawContours(_image, [ai_contour], 0, color=1, thickness=-1)

                dsc_li=[]
                for gt_contour in gt_contours:
                    _image = np.zeros_like(gt_im)
                    _image_gt = cv2.drawContours(_image, [gt_contour], 0, color=1, thickness=-1)
                    dsc_li.append(dice_coef_score(_image_gt, _image_ai))

            if max(dsc_li)>0:
                if name == 'brmh':
                    dscs1.append(max(dsc_li))
                elif name=='gugh':
                    dscs2.append(max(dsc_li))
                elif name=='kyuh':
                    dscs3.append(max(dsc_li))


brmh_respiratory
kyuh_respiratory
gugh_respiratory


In [ ]:
line=['-','-.','--']
plt.rcParams['font.family']='Times New Roman'

for j, dscs in enumerate([dscs1, dscs2, dscs3]):
    plt.figure(1)
    n, bins, patches = plt.hist(dscs, bins=5)
    plt.clf()

    x=[0]
    for i in range(len(bins)-1):
        a=bins[i]
        b=bins[i+1]
        x.append((a+b)/2)
    x.append(1) 

    y=np.append([0], n)
    y=np.append(y, [0])


    x=np.array(x)
    xnew = np.linspace(x.min(), x.max(), 200) 

    #define spline
    spl = make_interp_spline(x, y, k=2)
    y_smooth = spl(xnew)
    
#     sns.distplot(dscs, bins=5)
#     plt.tight_layout()
#     plt.show()
    
#     plt.plot(x, y)
#     plt.xlim([0,1])
#     plt.ylim([0,max(n)+20])
#     plt.show()

#     #create smooth line chart 
    if j == 0:
        plt.figure(2)
        plt.plot(xnew, y_smooth, line[j], color='k',linewidth=2,label='BRMH')
    elif j==1:
        plt.figure(2)
        plt.plot(xnew, y_smooth, line[j],color='k',linewidth=2, label='GUGH')
    elif j==2:
        plt.figure(2)
        plt.plot(xnew, y_smooth, line[j], color='k',linewidth=2,label='KYUH')
#     plt.clf()
plt.xlim([0,1])
plt.ylim([0,400])
plt.legend(loc='upper left', fontsize=15)
plt.xlabel('Dice Similarity Coefficient', fontsize=15)
plt.ylabel('TP count', fontsize=15)
plt.savefig('all_fig.tif', dpi=300)
plt.show()